In [1]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Model
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.metrics import Metric
from tensorflow.python.keras.utils import metrics_utils
from tensorflow.python.ops import init_ops, math_ops
from tensorflow.python.keras.utils.generic_utils import to_list
import tensorflow_hub as hub
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score  
import seaborn as sns
import codecs
import warnings 
warnings.filterwarnings("ignore") 
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.sparse import coo_matrix, hstack, vstack

import re
from nltk.corpus import stopwords

In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.set_index("ID", inplace=True)


In [4]:
def data_text_preprocess(total_text, ind, col):
    # Remove int values from text data as that might not be imp
    if type(total_text) is not int:
        string = ""
        # replacing all special char with space
        total_text = re.sub('[^a-zA-Z0-9\n]', ' ', str(total_text))
        # replacing multiple spaces with single space
        total_text = re.sub('\s+',' ', str(total_text))
        # bring whole text to same lower-case scale.
#         total_text = total_text.lower()
        
        for word in total_text.split():
        # if the word is a not a stop word then retain that word from text
            #if not word in stop_words:
            string += word + " "
        
        df[col][ind] = string

In [5]:
for index, row in df.iterrows():
    if type(row['ABSTRACT']) is str:
        data_text_preprocess(row['ABSTRACT'], index, 'ABSTRACT')
    if type(row['TITLE']) is str:
        data_text_preprocess(row['TITLE'], index, 'TITLE')
df.head(10)

,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
ID,,,,,,,,
1,Reconstructing Subject Specific Effect Maps,Predictive models allow subject specific infer...,1,0,0,0,0,0
2,Rotation Invariance Neural Network,Rotation invariance and translation invariance...,1,0,0,0,0,0
3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spheric...,0,0,1,0,0,0
4,A finite element approximation for the stochas...,The stochastic Landau Lifshitz Gilbert LLG equ...,0,0,1,0,0,0
5,Comparative study of Discrete Wavelet Transfor...,Fourier transform infra red FTIR spectra of sa...,1,0,0,1,0,0
6,On maximizing the fundamental frequency of the...,Let Omega subset mathbb R n be a bounded domai...,0,0,1,0,0,0
7,On the rotation period and shape of the hyperb...,We observed the newly discovered hyperbolic mi...,0,1,0,0,0,0
8,Adverse effects of polymer coating on heat tra...,The ability of metallic nanoparticles to suppl...,0,1,0,0,0,0
9,SPH calculations of Mars scale collisions the ...,We model large scale approx 2000km impacts on ...,0,1,0,0,0,0


In [6]:

train, test = train_test_split(df, random_state=42, test_size=0.2, shuffle=True)
# train.shape, test.shape
# test = pd.read_csv("cleaned_test_data.csv")

In [7]:

train_words_title, test_words_title = set(), set()
train_words_abstract, test_words_abstract = set(), set()
for index, row in train.iterrows():
    for  i in row['ABSTRACT'].split():
        train_words_abstract.add(i)
    for i in row['TITLE'].split():
        train_words_title.add(i)

for index, row in test.iterrows():
    for  i in row['ABSTRACT'].split():
        test_words_abstract.add(i)
    for i in row['TITLE'].split():
        test_words_title.add(i)
len(train_words_abstract - test_words_abstract), len(train_words_title - test_words_title)

(35602, 14917)

In [8]:
len(train_words_abstract), len(test_words_abstract), len(train_words_title), len(test_words_title)

(56894, 28093, 22574, 10381)

In [9]:
# w2i_title, w2i_abstract = {}, {}
# cnt = 0
# for i in train_words_abstract:
#     w2i_abstract.add()

In [10]:
# classifiers = {
#     "LogisiticRegression": LogisticRegression(),
#     "KNearest": KNeighborsClassifier(),
#     "Support Vector Classifier": SVC(),
#     "DecisionTreeClassifier": DecisionTreeClassifier(),
#     "MultinimialNB": MultinomialNB()
# }

In [11]:
abstract_max_len = 3000
title_max_len = 300
def get_label(row):
    label = [0, 0, 0, 0, 0, 0]
    columns = ['Computer Science', 'Physics', 'Mathematics',
       'Statistics', 'Quantitative Biology', 'Quantitative Finance']
    for col in range(len(columns)):
        if row[columns[col]] == 1:
            label[col] = 1
    return np.array(label)

def get_data(dframe):
#     X_title_vectors = count_vectorizer.transform(dframe['TITLE'])
#     X_abstract_vectors = count_vectorizer2.transform(dframe['ABSTRACT'])
#     X_title_vectors = hstack([X_title_vectors, X_abstract_vectors]).toarray()
    X_dframe = []
    y_dframe = []
    for index, row in tqdm(dframe.iterrows()):
        new_abstract_seq = []
        new_title_seq = []
        abstract = row["ABSTRACT"].split()
        title = row["TITLE"].split()
        for i in range(abstract_max_len):
            try: new_abstract_seq.append(abstract[i])
            except: new_abstract_seq.append("__PAD__")
        
        for i in range(title_max_len):
            try: new_title_seq.append(title[i])
            except: new_title_seq.append("__PAD__")
        y_dframe.append(get_label(row))        
        X_dframe.append([new_abstract_seq, new_title_seq])
    y_dframe = np.array(y_dframe)
    print((len(X_dframe), len(X_dframe[0])), y_dframe.shape)
    return X_dframe, y_dframe

In [12]:
X, y = get_data(train)
X_test, y_test = get_data(test)

16777it [00:17, 978.09it/s] 
101it [00:00, 1003.20it/s]

(16777, 2) (16777, 6)


4195it [00:04, 925.39it/s] 

(4195, 2) (4195, 6)


In [13]:
batch_size = 32

In [14]:
sess = tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session(sess)

In [15]:
def ElmoEmbedding1(x):
    return elmo_model(inputs={
                            "tokens": tf.squeeze(tf.cast(x, tf.string)),
                            "sequence_len": tf.constant(batch_size*[abstract_max_len])
                      },
                      signature="tokens",
                      as_dict=True)["elmo"]
def ElmoEmbedding2(x):
    return elmo_model(inputs={
                            "tokens": tf.squeeze(tf.cast(x, tf.string)),
                            "sequence_len": tf.constant(batch_size*[title_max_len])
                      },
                      signature="tokens",
                      as_dict=True)["elmo"]

In [16]:
tf.compat.v1.disable_eager_execution()
elmo_model = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)
sess.run(tf.compat.v1.global_variables_initializer())
sess.run(tf.compat.v1.tables_initializer())

In [17]:
input_text1 = Input(shape=(abstract_max_len,), dtype=tf.string)
embedding1 = Lambda(ElmoEmbedding1, output_shape=(None, 1024))(input_text1)
# input_text2 = Input(shape=(title_max_len,), dtype=tf.string)
# embedding2 = Lambda(ElmoEmbedding2, output_shape=(None, 1024))(input_text2)
# embedding = concatenate([embedding1, embedding2])
x = Bidirectional(LSTM(units=64, return_sequences=True,
                       ))(embedding1)
x_rnn = Bidirectional(LSTM(units=64, return_sequences=True,
                           ))(x)
# x = add([x, x_rnn])  # residual connection to the first biLSTM
out = Dense(6, activation="softmax")(x_rnn)

model = Model(input_text1, out)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [18]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3000)]            0         
_________________________________________________________________
lambda (Lambda)              (32, None, 1024)          0         
_________________________________________________________________
bidirectional (Bidirectional (32, None, 128)           557568    
_________________________________________________________________
bidirectional_1 (Bidirection (32, None, 128)           98816     
_________________________________________________________________
dense (Dense)                (32, None, 6)             774       
Total params: 657,158
Trainable params: 657,158
Non-trainable params: 0
_________________________________________________________________


In [20]:
X1 = [X[i][0] for i in range(len(X))]
X1 = X1[:524 * batch_size]
y1 = y[:524 * batch_size]
history = model.fit(np.array(X1), y1,
                    batch_size=batch_size, epochs=2, verbose=1)

Train on 16768 samples
Epoch 1/2


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[32,128,3000,47] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node lambda/module_apply_tokens/bilm/CNN/Conv2D_3}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[metrics/accuracy/Identity/_321]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[32,128,3000,47] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node lambda/module_apply_tokens/bilm/CNN/Conv2D_3}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

0 